In [ ]:
import numpy as np
import cv2 as cv
from ultralytics import YOLO


In [ ]:
from IPython import display
from IPython.display import display, Image
import glob

In [ ]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)


In [ ]:
torch.cuda.is_available()

In [ ]:
model = YOLO("yolov8s.pt")
model.info()

In [ ]:
results = model.train(data="annotated/data.yaml", epochs=60, imgsz=640)

In [ ]:
Image(filename=f'runs/detect/train6/confusion_matrix.png', width=800)

In [ ]:
Image(filename=f'runs/detect/train6/results.png', width=800)

In [ ]:
!yolo task=detect mode=val model=runs/detect/train6/weights/best.pt data=annotated/data.yaml

In [ ]:
!yolo task=detect mode=predict model=runs/detect/train6/weights/best.pt data=annotated/data.yaml source=annotated/test/images/

In [ ]:
for image_path in glob.glob(f'runs/detect/predict/*.jpg'):
    display(Image(filename=image_path, height=600))
    print("\n")

In [ ]:
trained_mahjong = YOLO('runs/detect/train6/weights/best.pt')

In [ ]:
results=model(source=0,show=True, conf=0.6)

from 
https://docs.opencv.org/4.x/dd/d43/tutorial_py_video_display.html

In [ ]:
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Cannot open camera")
    exit()

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
 
    # if frame is read correctly ret is True
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break
    # Our operations on the frame come here
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    # Display the resulting frame
    cv.imshow('frame', gray)
    if cv.waitKey(1) == ord('q'):
        break
 
# When everything done, release the capture
cap.release()
cv.destroyAllWindows()